In [11]:
library('igraph')
library('dplyr')

<h2>Question 6</h2>

In [12]:
movement_graph <- read.table('../uber_data/movement_edgelist.txt')

In [13]:
g <- graph_from_data_frame(movement_graph, directed=FALSE)

In [14]:
E(g)$weight=as.numeric(movement_graph[,3])

In [15]:
is.connected(g)
writeLines(paste("Number of edges: ", ecount(g)))
writeLines(paste("Number of vertices: ", vcount(g)))
writeLines(paste("Diameter: ", diameter(g)))
#plot(g, vertex.label=NA, vertex.size=3)

[1] FALSE

Number of edges:  311812
Number of vertices:  1887
Diameter:  11911.84


In [17]:
dg <- decompose.graph(g)      
gcc_index <- which.max(sapply(dg,vcount))
gcc <- dg[[gcc_index]] 
clusters <- components(gcc)
#print(dim(groups(clusters)))
writeLines(paste("Number of edges: ", ecount(gcc)))
writeLines(paste("Number of vertices: ", vcount(gcc)))
writeLines(paste("Diameter of GCC: ", diameter(gcc)))

Number of edges:  311802
Number of vertices:  1880
Diameter of GCC:  11911.84


In [18]:
geo_data <- read.table('../uber_data/geo_data.csv', sep=',', header=TRUE)

In [19]:
#print(which(name_sectors_colors$Symbol == 'ZTS'))
print(geo_data[which(geo_data$movement_id == '1'),1])

[1] Sargent Creek, San Ardo
2704 Levels: 0 12th East Street, Pittsburg ...


In [21]:
v = 1
which(V(gcc)$name == v)

integer(0)

In [22]:
v = 6
print(v)
gcc <- set_vertex_attr(gcc, "location", index=which(V(gcc)$name == v), as.character(geo_data[which(geo_data$movement_id == v), 2]))
gcc <- set_vertex_attr(gcc, "display_name", index=which(V(gcc)$name == v), as.character(geo_data[which(geo_data$movement_id == v), 1]))

[1] 6


In [23]:
geo_data[which(geo_data$movement_id == v), 1]

[1] 100 Carlsbad Circle, Vacaville
2704 Levels: 0 12th East Street, Pittsburg ...

In [24]:
vertex_attr(gcc, index=1)

$name
[1] "6"

$location
[1] "[-121.94843241   38.36665327]"

$display_name
[1] "100 Carlsbad Circle, Vacaville"

In [25]:
for (v in V(gcc)$name) {
    gcc <- set_vertex_attr(gcc, "location", index=which(V(gcc)$name == v), 
                           as.character(geo_data[which(geo_data$movement_id == v), 2]))
    gcc <- set_vertex_attr(gcc, "display_name", index=which(V(gcc)$name == v), 
                           as.character(geo_data[which(geo_data$movement_id == v), 1]))
}

In [26]:
all_info <- cbind(vertex_attr(gcc)$name, vertex_attr(gcc)$location, vertex_attr(gcc)$display_name)

<h2>Question 7</h2>

In [27]:
g_mst <- mst(gcc, algorithm="prim")

In [28]:
is.connected(g_mst)
writeLines(paste("Number of edges: ", ecount(g_mst)))
writeLines(paste("Number of vertices: ", vcount(g_mst)))
writeLines(paste("Diameter: ", diameter(g_mst)))
#plot(g, vertex.label=NA, vertex.size=3)

[1] TRUE

Number of edges:  1879
Number of vertices:  1880
Diameter:  45542.45


In [29]:
E(g_mst)

+ 1879/1879 edges from dc6cb20 (vertex names):
 [1] 7  --25  20 --79  43 --101 55 --102 57 --102 64 --104 62 --105 13 --111
 [9] 138--8   93 --150 123--153 43 --191 63 --219 198--223 178--224 230--115
[17] 39 --253 141--254 252--274 171--282 216--290 74 --300 143--301 78 --302
[25] 303--304 251--309 210--311 274--312 286--323 221--328 199--331 329--331
[33] 41 --334 91 --338 315--338 309--349 6  --366 275--370 253--372 369--372
[41] 325--382 333--389 175--396 371--397 270--400 40 --401 242--403 89 --420
[49] 30 --422 291--423 59 --435 134--437 80 --445 113--445 288--447 58 --456
[57] 283--456 98 --458 321--458 167--466 333--468 290--470 393--470 103--472
[65] 104--472 218--473 382--473 385--477 225--480 485--486 416--488 188--505
[73] 516--517 339--530 394--530 299--532 434--536 113--537 233--543 542--543
+ ... omitted several edges

In [30]:
vertex_attr(g_mst, index=which(V(g_mst)$name == 7))

$name
[1] "7"

$location
[1] "[-121.99610391   38.32584685]"

$display_name
[1] "Unnamed Road, Vacaville"

In [31]:
vertex_attr(g_mst, index=which(V(g_mst)$name == 25))

$name
[1] "25"

$location
[1] "[-121.9796351    38.34320795]"

$display_name
[1] "400 Bowline Drive, Vacaville"

<h2>Question 8</h2>

In [32]:
#graph_triangles = cliques(g, min=3,max=3)
graph_triangles = as.vector(triangles(gcc))

In [66]:
length(graph_triangles)/3

[1] 31678905

In [78]:
set.seed(0)
sample_ids = sample(length(graph_triangles)/3, 1000)

In [33]:
get_id <- function(x) {
    return (which(V(gcc)$name == x$name))
}

In [59]:
check_triangular_inequality <- function(w1, w2, w3) {
    if (((w1 + w2) >= w3) & ((w1 + w3) >= w2) & ((w3 + w2) >= w1)) {
        return (1)
    } else {
        return (0)
    }
}

In [79]:
triads = c()
for (s in sample_ids) {
    #print(i)
    i = s*3 + 1
    u = (graph_triangles[[i]])
    v = (graph_triangles[[i+1]])
    w = (graph_triangles[[i+2]])
    
    e1 <- get.edge.ids(gcc, vp=c(u,v))
    e2 <- get.edge.ids(gcc, vp=c(v,w))
    e3 <- get.edge.ids(gcc, vp=c(u,w))
    #ei <- get.edge.ids(gcc, vp=lapply(c(u, v, v, w, u, w), get_name))
    w1 = edge_attr(gcc, "weight", index=e1)
    w2 = edge_attr(gcc, "weight", index=e2)
    w3 = edge_attr(gcc, "weight", index=e3)
    #print(c(u,v,w))
    triads = c(triads, check_triangular_inequality(w1, w2, w3))
}

In [75]:
tri_eq = sum(triads)/length(triads)

In [76]:
length(triads)

[1] 1000

In [77]:
writeLines(paste("Percentage of triangles satisfying triangular inequality: ", tri_eq))

Percentage of triangles satisfying triangular inequality:  0.902


<h2>Question 9</h2>